# Treinamento com Neural Networks

In [89]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split

In [90]:
url='https://drive.google.com/file/d/1GXJpU1eAKFoGABsy2CnfaP5dSHaoxHTY/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
raw_dataset = pd.read_csv(url)

features = [
    'win', 'firstBlood', 'firstTower', 'firstInhibitor',
    'firstBaron', 'firstDragon', 'firstRiftHerald', 'towerKills',
    'inhibitorKills', 'baronKills', 'dragonKills', 'riftHeraldKills',
    'kills', 'deaths', 'assists', 'visionScore', 'csPerMin',
    'goldPerMin', 'crowdControlTime'
]

to_drop_red  = ['red.' + feature for feature in features]
to_drop_blue = ['blue.' + feature for feature in features]

blue_dataset = raw_dataset.copy()
blue_dataset.drop(to_drop_red, axis=1, inplace=True)

red_dataset = raw_dataset.copy()
red_dataset.drop(to_drop_blue, axis=1, inplace=True)


In [91]:
def split_90_10(df, target_name):
    target = df[target_name]
    features = df.drop(target_name, 1)
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=42)
    return X_train, X_test, y_train, y_test

def to_np(X_train, X_test, y_train, y_test):
    X_train = X_train.to_numpy()
    y_train = y_train.to_numpy()
    X_test  = X_test.to_numpy()
    y_test  = y_test.to_numpy()
    return X_train, X_test, y_train, y_test

In [92]:
X_train, X_test, y_train, y_test = split_90_10(blue_dataset, 'blue.win')
X_train_np, X_test_np, y_train_np, y_test_np = to_np(X_train, X_test, y_train, y_test)

X_train_np.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


(14644, 18)

# O modelo da rede neural

A rede neural é feita sobre o modelo sequencial do Keras e é constituída em: 

* Input Layer de 18 features.
* 3 Hidden Layers com ativação *selu*.
* Output Layer com apenas um neurônio (derrota = 0 | vitória = 1)

Além disso, a função de perda é definida como *binary crossentropy* e o otimizador é *Adam*.

# Treinamento da rede neural

Para treinar o modelo, os dados foram separados em 10% para teste e 90% para treino, com 10% destes para validação. Utilizamos a técnica de *Early Stopping* baseada no valor da função de perda no conjunto de validação e por isso que o número de épocas é alto.

Quanto ao tamanho do batch, escolhemos 32 pois o tempo de computação é razoável e garante resultados com mais precisão.

In [93]:
def neural_network(verbose=False):
  # Create a model
  model = tf.keras.Sequential()

  # Add layers
  model.add(tf.keras.layers.Flatten(input_shape=(18,)))
  model.add(tf.keras.layers.Dense(10, activation='selu'))
  model.add(tf.keras.layers.Dense(10, activation='selu'))
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

  # Fit model with early stopping
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8)
  model.fit(X_train_np, y_train_np, epochs=50, batch_size=32, validation_split=0.1,
            callbacks=[callback], verbose=verbose)

  # Test accuracy
  if verbose: print('\n')
  _, accuracy = model.evaluate(X_test_np, y_test_np)
  return accuracy*100

# Acurácia

In [94]:
neural_network(verbose=True)

Epoch 1/50
412/412 [==============================] - 2s 3ms/step - loss: 4.4735 - accuracy: 0.6136 - val_loss: 0.3490 - val_accuracy: 0.9522
Epoch 2/50
412/412 [==============================] - 1s 3ms/step - loss: 0.2404 - accuracy: 0.9524 - val_loss: 0.1769 - val_accuracy: 0.9625
Epoch 3/50
412/412 [==============================] - 1s 2ms/step - loss: 0.1457 - accuracy: 0.9638 - val_loss: 0.1107 - val_accuracy: 0.9706
Epoch 4/50
412/412 [==============================] - 1s 3ms/step - loss: 0.1139 - accuracy: 0.9670 - val_loss: 0.1068 - val_accuracy: 0.9706
Epoch 5/50
412/412 [==============================] - 1s 3ms/step - loss: 0.0976 - accuracy: 0.9694 - val_loss: 0.0788 - val_accuracy: 0.9720
Epoch 6/50
412/412 [==============================] - 1s 3ms/step - loss: 0.0978 - accuracy: 0.9674 - val_loss: 0.0982 - val_accuracy: 0.9666
Epoch 7/50
412/412 [==============================] - 1s 3ms/step - loss: 0.0924 - accuracy: 0.9674 - val_loss: 0.1022 - val_accuracy: 0.9625
Epoch 

92.32186675071716

# Resultados

**A rede neural apresenta acurácia média de 96.6%.**

Abaixo está uma pequena confirmação de que o resultado da rede não foi escolhido à mão.

In [95]:
r = 10
acc = [neural_network() for _ in range(r)]
print(f'Mean: {np.mean(acc):.3f}% | Std: {np.std(acc):.3f}%')

51/51 [==============================] - 0s 2ms/step - loss: 0.1067 - accuracy: 0.9564
Mean: 96.615% | Std: 0.676%
